- conda 安裝 python 3.11 版本
- conda install ipykernel
- sudo apt update
- sudo apt install ffmpeg
- ffprobe -version
- 把 keras_model.py 的 DepthwiseConv2D 裡的 kernel_regularizer 改成 depthwise_regularizer

In [ ]:
!pip install pydub tensorflow[and-cuda] matplotlib tensorflow-datasets

In [ ]:
# 生成 kws_model.h5
!python3 train.py

2024-12-11 15:53:32.742095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733903612.758292  137400 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733903612.763105  137400 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 15:53:32.778811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
We will download data to /home/kycho/data
We will train for 36 epochs
I0000 00:00:1733903614.545787  137400 gpu_devic

In [ ]:
# 生成 kws_model.tflite 以及 kws_model_float32.tflite
!python3 quantize.py

2024-12-11 16:14:55.315553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733904895.330944  146203 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733904895.335383  146203 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 16:14:55.350467: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Converting trained model trained_models/kws_model.h5 to TFL model at trained_models/kws_model.tflite
I0000 00:00:1733

In [ ]:
!ls -lt trained_models/

total 656
-rw-r--r-- 1 kycho gpu-user  47536 Dec 11 16:15 kws_model.tflite
-rw-r--r-- 1 kycho gpu-user  45544 Dec 11 16:15 kws_model_float32.tflite
-rw-r--r-- 1 kycho gpu-user 454912 Dec 11 16:01 kws_model.h5
-rw-r--r-- 1 kycho gpu-user  43392 Dec 11 12:05 kws_ref_model_float32.tflite
drwxr-xr-x 3 kycho gpu-user   4096 Dec 11 12:05 kws_ref_model
-rw-r--r-- 1 kycho gpu-user  53936 Dec 11 12:05 kws_ref_model.tflite
-rw-r--r-- 1 kycho gpu-user    556 Dec 11 12:05 README.md
-rw-r--r-- 1 kycho gpu-user     55 Dec 11 12:05 kws_ref_model.md5


In [ ]:
# 生成 kws_model.tflite and kws_model_float32.tflite
!python3 eval_quantized_model.py

2024-12-11 16:25:42.110431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733905542.125500  147834 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733905542.129770  147834 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 16:25:42.144622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1733905544.283715  147834 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0

In [8]:
!ls -lt trained_models/

total 656
-rw-r--r-- 1 kycho gpu-user  47536 Dec 11 16:15 kws_model.tflite
-rw-r--r-- 1 kycho gpu-user  45544 Dec 11 16:15 kws_model_float32.tflite
-rw-r--r-- 1 kycho gpu-user 454912 Dec 11 16:01 kws_model.h5
-rw-r--r-- 1 kycho gpu-user  43392 Dec 11 12:05 kws_ref_model_float32.tflite
drwxr-xr-x 3 kycho gpu-user   4096 Dec 11 12:05 kws_ref_model
-rw-r--r-- 1 kycho gpu-user  53936 Dec 11 12:05 kws_ref_model.tflite
-rw-r--r-- 1 kycho gpu-user    556 Dec 11 12:05 README.md
-rw-r--r-- 1 kycho gpu-user     55 Dec 11 12:05 kws_ref_model.md5


把
```
sed -i .bak '1 s/^/#include "kws_model_data.h"\n\n/'  kws_model_data.cc
sed -i .bak 's/unsigned char/const unsigned char/' kws_model_data.cc
sed -i .bak "s/$FNAME_CONV/g_kws_model_data/" kws_model_data.cc
sed -i .bak 's/unsigned int/const unsigned int/' kws_model_data.cc
```
改成
```
sed -i '1 s/^/#include "kws_model_data.h"\n\n/'  kws_model_data.cc
sed -i 's/unsigned char/const unsigned char/' kws_model_data.cc
sed -i "s/$FNAME_CONV/g_kws_model_data/" kws_model_data.cc
sed -i 's/unsigned int/const unsigned int/' kws_model_data.cc
```

In [14]:
# Convert to C++ Code
!./tflm2cc trained_models/kws_model.tflite

Converting file trained_models/kws_model.tflite to file kws_model_data.cc


In [ ]:
# 生成 kws_model_data.cc
!ls -lt

total 468
-rw-r--r-- 1 kycho gpu-user  58105 Dec 11 16:36 __0.ipynb
-rw-r--r-- 1 kycho gpu-user 293263 Dec 11 16:36 kws_model_data.cc
-rwxr-xr-x 1 kycho gpu-user    850 Dec 11 16:35 tflm2cc
-rw-r--r-- 1 kycho gpu-user    119 Dec 11 16:26 __1.ipynb
drwxr-xr-x 3 kycho gpu-user   4096 Dec 11 16:15 trained_models
drwxr-xr-x 2 kycho gpu-user   4096 Dec 11 16:01 plots
drwxr-xr-x 2 kycho gpu-user   4096 Dec 11 15:53 __pycache__
-rw-r--r-- 1 kycho gpu-user   9117 Dec 11 15:53 keras_model.py
-rw-r--r-- 1 kycho gpu-user   4701 Dec 11 12:05 README.md
-rw-r--r-- 1 kycho gpu-user    401 Dec 11 12:05 build_ref.sh
-rw-r--r-- 1 kycho gpu-user   8694 Dec 11 12:05 eval_functions_eembc.py
-rw-r--r-- 1 kycho gpu-user   2654 Dec 11 12:05 eval_quantized_model.py
-rw-r--r-- 1 kycho gpu-user   1949 Dec 11 12:05 evaluate.py
-rw-r--r-- 1 kycho gpu-user  15447 Dec 11 12:05 get_dataset.py
-rw-r--r-- 1 kycho gpu-user   6374 Dec 11 12:05 kws_util.py
-rw-r--r-- 1 kycho gpu-user    503 Dec 11 12:05 make_all_bin_files

In [1]:
!mv kws_model_data.cc ../../reference_submissions/keyword_spotting/kws/